# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.  
 
Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.probability import ConditionalFreqDist

In [2]:
import tkinter as tk
from tkinter.filedialog import askopenfilename
import pandas as pd

root = tk.Tk()
root.withdraw() #Prevents the Tkinter window to come up
exlpath = askopenfilename()
root.destroy()
print(exlpath)

C:/Users/franc/Desktop/Labs Iron Hack/Last labs 16.07.2022/lab-nlp/training_dataset.csv


In [3]:
# your code here
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    string = re.sub(r'http\S+', '', s)
    return re.sub('[^A-Za-z]+', ' ', string).lower().strip()

def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    return nltk.word_tokenize(s)

def stem_and_lemmatize(l):
    
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    ps = nltk.PorterStemmer()
    lemmatizer = nltk.WordNetLemmatizer()
    l2 = []
    
    for w in l:
        s = ps.stem(w)
        s = lemmatizer.lemmatize(s)
        l2 += [s]
    
    return l2


def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    stop_words = stopwords.words('english')

    return [w for w in l if w not in stop_words]

In [4]:
tweets = pd.read_csv(exlpath, engine='python',encoding="ISO-8859-1",names=['Index_0', 'DATE', 'QUERY', 'User','text'], header=None)

In [5]:
tweets.head()

,Index_0,DATE,QUERY,User,text
0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
sample = tweets.sample(20000)
sample['target'] = sample['Index_0'].replace(4, 1)

In [7]:
sample.head()

,Index_0,DATE,QUERY,User,text,target
0,1975865841,Sat May 30 16:04:18 PDT 2009,NO_QUERY,martinsandoval,I have so much grains... I'm scared,1975865841
0,2298482191,Tue Jun 23 11:44:01 PDT 2009,NO_QUERY,drunkenscholar,@LaurenWJohnston There's 1 copy and 5 requests.,2298482191
0,1882734354,Fri May 22 07:23:40 PDT 2009,NO_QUERY,SophieCollinss,"is eating maltesers, have work at 5",1882734354
4,1880764926,Fri May 22 02:25:19 PDT 2009,NO_QUERY,mrQQ,I'm in love with fairytale.. and it doesnt hur...,1880764926
4,2066608145,Sun Jun 07 10:50:25 PDT 2009,NO_QUERY,enithhernandez,.@MarcelloJun or you are just a sheeple?,2066608145


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [8]:
# your code here
sample["text_processed"]= sample["text"].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)

### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [9]:
# your code here
count_freq= nltk.FreqDist()

for tweet in sample["text_processed"]:
    for word in tweet:
       count_freq[word]+=1

top_5k_words= list(count_freq.keys())[:5000]
top_5k_words

['much',
 'grain',
 'scare',
 'laurenwjohnston',
 'copi',
 'request',
 'eat',
 'maltes',
 'work',
 'love',
 'fairytal',
 'doesnt',
 'hurt',
 'marcellojun',
 'sheepl',
 'sadli',
 'chang',
 'prize',
 'sorri',
 'still',
 'conquest',
 'bad',
 'scynet',
 'thank',
 'rt',
 'mention',
 'galtim',
 'get',
 'use',
 'thi',
 'phone',
 'tri',
 'twitterfon',
 'move',
 'stuff',
 'oh',
 'miss',
 'old',
 'risarm',
 'urgh',
 'ad',
 'tv',
 'australia',
 'say',
 'quot',
 'u',
 'dont',
 'like',
 'chicken',
 'someth',
 'wrong',
 'disgust',
 'spent',
 'last',
 'night',
 'clean',
 'fish',
 'tank',
 'morn',
 'mop',
 'overnight',
 'leak',
 'spend',
 'next',
 'hour',
 'buy',
 'new',
 'nbc',
 'even',
 'read',
 'stori',
 'thnx',
 'follow',
 'alauderdal',
 'gurumonet',
 'bum',
 'three',
 'plan',
 'got',
 'cancel',
 'today',
 'urg',
 'steve',
 'job',
 'liver',
 'transplant',
 'doe',
 'mean',
 'metastat',
 'cancer',
 'appl',
 'ha',
 'workingipod',
 'lie',
 'around',
 'longer',
 'borrow',
 'neither',
 'mine',
 'order',

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [10]:

# your code here
def find_features(document):
    words = set(document)
    features = {}
    for w in top_5k_words:
        features[w] = (w in words)
    return features

features_set= [(find_features(tweet), target) for (tweet, target) in list(zip(sample['text_processed'], sample['target']))]
print(len(features_set))

20000


### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [11]:
# your code here

from textblob.classifiers import NaiveBayesClassifier
train_set,test_set = features_set[:10000],features_set[10000:]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
classifier.show_most_informative_features()

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here